In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize language model
llm = ChatOpenAI(
        api_key="ollama",
        model="llama3:latest",
        #model="gemma2:latest", #respostas sempre em ingles
        #model="splitpierre/bode-alpaca-pt-br:latest", 
        #model="splitpierre/bode-alpaca-pt-br:13b-Q4_0", 
        base_url="http://localhost:11434/v1",
        temperature=0, 
    )

In [13]:
from pydantic import BaseModel, Field, field_validator
from typing import List


In [11]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @field_validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field
    
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt_template = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt_template | llm | parser

res = chain.invoke(joke_query)

print(res)


In [30]:
from langchain import LLMChain

# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

parser = PydanticOutputParser(pydantic_object=Actor)

prompt_template = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

#chain = prompt_template | llm | parser
chain = LLMChain(llm=llm, prompt=prompt_template)

actor_query = "Generate the filmography for a random actor."
#actor_query = {'query':'Generate the filmography for Will Smith.'}
#actor_query = {'query': 'Generate the filmography for Will Smith.'}

# Run the LLMChain to get the AI-generated answer
output = chain.run(actor_query)
print( output )
#parser.parse(output)




Here is the output in JSON format:

```
{
    "properties": {
        "name": {"description": "name of an actor", "title": "Name", "type": "string"},
        "film_names": {"description": "list of names of films they starred in", "items": {"type": "string"}, "title": "Film Names", "type": "array"}
    },
    "required": ["name", "film_names"]
}
```

Here is the output:

```
{
    "name": "Tom Hanks",
    "film_names": [
        "Forrest Gump",
        "Cast Away",
        "Saving Private Ryan",
        "Apollo 13",
        "The Green Mile"
    ]
}
```


In [31]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of recieving a numbered-list from API
    @field_validator('words')
    def not_start_with_number(cls, field):
        if field[0].isnumeric():
            raise ValueError("The word can not start with numbers!")
        return field
    
parser = PydanticOutputParser(pydantic_object=Suggestions)

template = """
    Offer a list of suggestions to substitue the specified target_word based the presented context.
    {format_instructions}
    target_word={target_word}
    context={context}
"""

target_word = "behaviour"
context = "The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."

prompt_template = PromptTemplate(
    template = template,
    input_variables = ["target_word", "context"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

#chain = LLMChain(llm=llm, prompt=prompt_template)
chain = prompt_template | llm | parser

# Run the LLMChain to get the AI-generated answer
#output = chain.run({"target_word": target_word, "context":context})
output = chain.invoke({"target_word": target_word, "context":context})
print( output )

#parser.parse(output)

Suggestions(words=['conduct', 'manners', 'attitude', 'mannerisms', 'disruptiveness'])